In [5]:
import sys
sys.path.insert(0, '../../../../fastshap_tf/')
from fastshap import ResizeMask

2022-11-10 15:49:41.187098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 15:49:41.418080: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 15:49:42.161378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium

In [6]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime
import os

In [8]:
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

In [9]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

## Load Data

In [10]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [11]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


2022-11-10 16:08:19.085094: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-10 16:08:19.085163: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gpu-instance
2022-11-10 16:08:19.085176: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gpu-instance
2022-11-10 16:08:19.085363: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.57.2
2022-11-10 16:08:19.085398: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.57.2
2022-11-10 16:08:19.085406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.57.2
2022-11-10 16:08:19.085986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

### Batch Data

In [12]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [13]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [14]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = '../model/20210514_14_39_21/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../model/20210514_14_39_21/model_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Train Evaluator

### Prepare Dataset w/ Model Predictions

In [11]:
@tf.function
def make_prediction_data(x, y):
    with tf.device("gpu:2"):
        y_model = model(x)
    
    return (x, y_model)

with tf.device("gpu:2"):
    ds_train_pred = ds_train.map(make_prediction_data)
    ds_val_pred = ds_val.map(make_prediction_data)
    ds_test_pred = ds_test.map(make_prediction_data)

### Save Dir

In [18]:
save_dir = 'evaluator'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Initialize Model

In [15]:
class Random_Bernoulli_Sampler(Layer):
    '''
    Layer to Sample r
    '''
    def __init__(self, num_features, **kwargs):
        super(Random_Bernoulli_Sampler, self).__init__(**kwargs)
        
        self.num_features = num_features
        
        
    def call(self,  logits):
        batch_size = tf.shape(logits)[0]
        
        u = tf.random.uniform(shape=(batch_size, self.num_features),
                                    minval=np.finfo(
                                        tf.float32.as_numpy_dtype).tiny,
                                    maxval=1.0) 

        r = K.cast(tf.stop_gradient(u > 0.5), tf.float32)
        
        return r

In [16]:
base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')
R = Random_Bernoulli_Sampler(14*14)(model_input)
R = ResizeMask(in_shape=INPUT_SHAPE, mask_size=14*14)(R)
xr = Multiply()([model_input, R])

net = base_model(xr)
out = Dense(10, activation='softmax')(net)

evaluator_model = Model(model_input, out)

### Train Model

In [19]:
LR = 1e-3

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

# Model Checkpointing
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=5, 
                             verbose=1, mode='min', cooldown=1, min_lr=1e-3/10)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=10) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

# Train Model
evaluator_model.fit(ds_train,
          epochs = 20,
          validation_data = ds_val,
          callbacks = CALLBACKS)

# Get Checkpointed Model
print(model_weights_path)
evaluator_model.load_weights(model_weights_path)
evaluator_model = Sequential(   
            [l for l in evaluator_model.layers[-2:]]
        )
evaluator_model.trainable = False

2022-11-11 03:21:06.003234: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/100


2022-11-11 03:21:13.511120: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.


1563/1563 [==============================] - ETA: 0s - loss: 1.1423 - auroc: 0.9257 - auprc: 0.6704 - accuracy: 0.5958
Epoch 1: val_loss improved from inf to 0.96486, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/evaluation/evaluator/model_weights.h5
1563/1563 [==============================] - 11187s 7s/step - loss: 1.1423 - auroc: 0.9257 - auprc: 0.6704 - accuracy: 0.5958 - val_loss: 0.9649 - val_auroc: 0.9468 - val_auprc: 0.7463 - val_accuracy: 0.6630 - lr: 0.0010
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 0.7439 - auroc: 0.9676 - auprc: 0.8326 - accuracy: 0.7404
Epoch 2: val_loss improved from 0.96486 to 0.83215, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/evaluation/evaluator/model_weights.h5
1563/1563 [==============================] - 8896s 6s/step - loss: 0.7439 - auroc: 0.9676 - auprc: 0.8326 - accuracy: 0.7404 - val_loss: 0.8321 - val_auroc: 0.9602 -

KeyboardInterrupt: 

In [21]:
# Evaluate
evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)
evaluator_model.evaluate(ds_test)

157/157 [==============================] - 238s 2s/step - loss: 0.5068 - auroc: 0.9980 - auprc: 0.9925 - accuracy: 0.9662


[0.5067557096481323,
 0.9980300068855286,
 0.9925033450126648,
 0.9662293791770935]

In [23]:
evaluator_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 0
Non-trainable params: 23,608,202
_________________________________________________________________
